In [13]:
import os, sys, glob, csv
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
import xlsxwriter
import random
from random import shuffle
from math import log, floor
import re
import collections
from collections import Counter
import string
import unicodedata as udata
# import pause, datetime
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from distutils.dir_util import copy_tree
import sklearn
from sklearn.metrics import *
import itertools as it
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk import word_tokenize
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import functools
import spacy
import swifter
from pandarallel import pandarallel

import tensorflow.keras.preprocessing.text as T
import torch
import dask.dataframe as dd
from dask.multiprocessing import get

read data

In [2]:
train_df = pd.read_csv('./data/20news-bydate-v3/training_df.csv',index_col=False)
train_df = train_df.drop(['Unnamed: 0'],axis=1)
valid_df = pd.read_csv('./data/20news-bydate-v3/validation_df.csv',index_col=False)
valid_df = valid_df.drop(['Unnamed: 0'],axis=1)
test_df = pd.read_csv('./data/20news-bydate-v3/test_df.csv',index_col=False)
test_df = test_df.drop(['Unnamed: 0'],axis=1)
train_df

,id,category,preprocess_text
0,19313,rec.sport.hockey,Kovalev is too talented a player to play for R...
1,19312,rec.sport.hockey,First of all the Penguins WILL win the cup aga...
2,19115,rec.sport.hockey,You ca n't But good luck trying
3,19224,rec.sport.hockey,As the subject suggests the Flames were not im...
4,19299,rec.sport.hockey,Well I got ta tell ya last night 's Leafs game...
...,...,...,...
10548,20797,alt.atheism,When the object of their belief is said to be ...
10549,20537,alt.atheism,Koff You mean that as long as I put you to sle...
10550,20757,alt.atheism,Mr Connor 's assertion that more complex later...
10551,20677,alt.atheism,excess stuff deleted I know of a similar incid...


category encode

In [3]:
categories = set(train_df.category.tolist())
print(len(categories))
cat_encode = {}
for i,cat in enumerate(categories):
    cat_encode[cat] = i
pickle.dump(obj=cat_encode,file=open('./data/20news-bydate-v3/cat_encode_dict.pkl','wb'))
cat_encode

20


{'talk.politics.misc': 0,
 'talk.politics.mideast': 1,
 'comp.sys.ibm.pc.hardware': 2,
 'rec.sport.baseball': 3,
 'sci.crypt': 4,
 'rec.autos': 5,
 'sci.med': 6,
 'comp.windows.x': 7,
 'sci.electronics': 8,
 'soc.religion.christian': 9,
 'rec.motorcycles': 10,
 'talk.politics.guns': 11,
 'comp.os.ms-windows.misc': 12,
 'alt.atheism': 13,
 'talk.religion.misc': 14,
 'comp.graphics': 15,
 'misc.forsale': 16,
 'comp.sys.mac.hardware': 17,
 'rec.sport.hockey': 18,
 'sci.space': 19}

In [4]:
def convert_cat(serie):
    '''
    trainY轉換為ID: 0~19 (20類)
    '''
    return cat_encode[serie]

In [5]:
train_df['cat_id'] = train_df.category.apply(convert_cat)
valid_df['cat_id'] = valid_df.category.apply(convert_cat)
test_df['cat_id'] = test_df.category.apply(convert_cat)
train_df

,id,category,preprocess_text,cat_id
0,19313,rec.sport.hockey,Kovalev is too talented a player to play for R...,18
1,19312,rec.sport.hockey,First of all the Penguins WILL win the cup aga...,18
2,19115,rec.sport.hockey,You ca n't But good luck trying,18
3,19224,rec.sport.hockey,As the subject suggests the Flames were not im...,18
4,19299,rec.sport.hockey,Well I got ta tell ya last night 's Leafs game...,18
...,...,...,...,...
10548,20797,alt.atheism,When the object of their belief is said to be ...,13
10549,20537,alt.atheism,Koff You mean that as long as I put you to sle...,13
10550,20757,alt.atheism,Mr Connor 's assertion that more complex later...,13
10551,20677,alt.atheism,excess stuff deleted I know of a similar incid...,13


In [6]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.cuda() #gpu
# roberta.cpu()
roberta.eval() 

Using cache found in /home/leoqaz12/.cache/torch/hub/pytorch_fairseq_master


loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz from cache at /home/leoqaz12/.cache/torch/pytorch_fairseq/83e3a689e28e5e4696ecb0bbb05a77355444a5c8a3437e0f736d8a564e80035e.c687083d14776c1979f3f71654febb42f2bb3d9a94ff7ebdfe1ac6748dba89d2
| dictionary: 50264 types


RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          )
          (1): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=Tru

In [7]:

def tokenize_text(preprocess_text,max_length=481):
    '''
    利用RoBERTa tokenizer轉換 (太耗記憶體)
    '''
    roberta_pad = [0]*1024
    doc = roberta.extract_features_aligned_to_words(preprocess_text)
    roberta_emb = []
    tokens_li = []
    for tok in doc:
        roberta_emb.append(tok.vector)
        tokens_li.append(str(tok))
    if len(roberta_emb)<max_length:
        for _ in range(max_length-len(roberta_emb)):
            roberta_emb.append(roberta_pad)
    return np.array(roberta_emb) , tokens_li

In [8]:
train_df['roberta_emb'] = 'NAN'
train_df['roberta_tok'] = 0
train_df.loc[0,'roberta_emb']=1
preprcoess_text_li = train_df.preprocess_text.tolist()
print(preprcoess_text_li[0])
train_df


Kovalev is too talented a player to play for Roger needs players who ca n't think for themselves and can only skate in straight lines up and down the ice Dudley and Nielson are the only two coaches bad enough in the league to take talents like Mogilny and Kovalev and not know how to turn them into Bure and Selanne Get Muckler as coach and Kovalev will look like Mogilny The trouble with the Rangers is that Neil Smith went out and got players like Messier Kovalev and Graves who have been schooled in taking the game to their opponent and attacking while hiring coaches who are interested in ropeadope strategies If you want the Roger Nielsons of the world to be your coach you do n't go out and get a Mark Messier who is an old dog who ca n't learn new tricks from a known loser and you do n't waste your draft picks on players like Kovalev Gerald


,id,category,preprocess_text,cat_id,roberta_emb,roberta_tok
0,19313,rec.sport.hockey,Kovalev is too talented a player to play for R...,18,1,0
1,19312,rec.sport.hockey,First of all the Penguins WILL win the cup aga...,18,NAN,0
2,19115,rec.sport.hockey,You ca n't But good luck trying,18,NAN,0
3,19224,rec.sport.hockey,As the subject suggests the Flames were not im...,18,NAN,0
4,19299,rec.sport.hockey,Well I got ta tell ya last night 's Leafs game...,18,NAN,0
...,...,...,...,...,...,...
10548,20797,alt.atheism,When the object of their belief is said to be ...,13,NAN,0
10549,20537,alt.atheism,Koff You mean that as long as I put you to sle...,13,NAN,0
10550,20757,alt.atheism,Mr Connor 's assertion that more complex later...,13,NAN,0
10551,20677,alt.atheism,excess stuff deleted I know of a similar incid...,13,NAN,0


In [9]:
preprcoess_text_li = train_df.preprocess_text.tolist()
roberta_pad = [0]*1024
max_length = 498 #改
train_df['roberta_emb'] = 0
train_df['roberta_tok'] = 0
# train_df['glove_emb'] = 0

train_df=train_df.astype(object)
# glove = pd.read_table('./data/preprocess/glove.840B.300d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
# def glove_convert(tok_li):
#     '''
#     Deprecated
#     Input: list of tokens by RoBERTa
#     Return: np array of max_length*300dim
#     '''
#     max_length = 498 #改
#     glove_pad = [0]*300
#     glove_emb = []
#     for token in tok_li:
#         glove_emb.append(glove.loc[token].values)
#     if len(glove_emb)<max_length:
#         for _ in range(max_length-len(glove_emb)):
#             glove_emb.append(glove_pad)
#     try:
#         assert len(glove_emb) == max_length
#     except:
#         print("maxlength ERR(now,ori):",len(glove_emb) , max_length)
#     return np.array(glove_emb)



6091it [07:00, 15.05it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
for i,text in tqdm(enumerate(preprcoess_text_li)):
    doc = roberta.extract_features_aligned_to_words(text)
    roberta_emb = []
    tokens_li = []
    for tok in doc:
        roberta_emb.append(tok.vector.cpu().data.numpy())
        tokens_li.append(str(tok))
    try:        
        assert len(tokens_li) == len(roberta_emb)
    except:
        print('AsserionERR: ',len(tokens_li), len(roberta_emb))
    if len(roberta_emb)<max_length:
        for _ in range(max_length-len(roberta_emb)):
            roberta_emb.append(roberta_pad)
    train_df.loc[i,'roberta_emb'] = np.array(roberta_emb)
    train_df.loc[i,'roberta_tok'] = tokens_li
#     train_df.loc[i,'glove_emb'] = glove_convert(tokens_li) #法1
    try:
        assert len(np.array(roberta_emb)) == (max_length)
    except:
        print('max length err(now max length,set max length):',len(np.array(roberta_emb)),(max_length))
train_df

embedding layer init matrix
* initial weight
* tokens #

In [ ]:
# embeddings_index = {}
# f = open('./data/preprocess/glove.840B.300d.txt','r')
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))

In [ ]:

glove = pd.read_table('./data/preprocess/glove.840B.300d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
glove.index = glove.index.astype('str')
glove_words = glove.index.tolist()
print('all_iters#:',len(glove_words))
glove_index = {}
embedding_matrix = np.zeros((len(glove_words) + 1, 300))


for i,word in tqdm(enumerate(glove_words)):
    glove_index[word] = i+1
    try:
        embedding_matrix[i+1] = glove.loc[str(word)].values
    except KeyError:
        continue
    except ValueError:
        embedding_matrix[i+1] = glove.loc[str(word)].values[0]


pickle.dump(obj=glove_index,file=open('./data/20news-bydate-v3/glove_index.pkl','wb'))
pickle.dump(obj=embedding_matrix,file=open('./data/20news-bydate-v3/embedding_matrix.pkl','wb'))


In [63]:
def tok2int(tok_li):
    '''
    Input: tok_list
    Return: int_list
    '''
    max_length = 498 #改
    int_pad = 0
    int_list = []
    for tok in tok_li:
        int_list.append(glove_index[str(tok)])
    if len(int_list)<max_length:
        for _ in range(max_length-len(tok_li)):
            int_list.append(int_pad)
    try:
        assert len(int_list) == max_length
    except:
        print("maxlength ERR(now,ori):",len(int_list) , max_length)            
    return int_list

train_df['glove_int'] = 0
train_df=train_df.astype(object)
train_df['glove_int'] = train_df['roberta_tok'].swifter.apply(tok2int)
train_df

/home/leoqaz12/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


,id,category,preprocess_text,cat_id,roberta_emb,roberta_tok,glove_emb,glove_int
0,19313,rec.sport.hockey,Kovalev is too talented a player to play for R...,18,"[[-0.1072988361120224, 0.006329931318759918, -...","[<s>, Kovalev, is, too, talented, a, player, t...",0,"[566229, 119013, 11, 188, 5061, 7, 983, 5, 331..."
1,19312,rec.sport.hockey,First of all the Penguins WILL win the cup aga...,18,"[[-0.0944674089550972, 0.004107087850570679, -...","[<s>, First, of, all, the, Penguins, WILL, win...",0,"[566229, 586, 6, 44, 3, 18799, 5777, 814, 3, 2..."
2,19115,rec.sport.hockey,You ca n't But good luck trying,18,"[[-0.05532881245017052, 0.02691790461540222, 0...","[<s>, You, ca, n't, But, good, luck, trying, <...",0,"[566229, 94, 324, 41, 175, 113, 2455, 553, 437..."
3,19224,rec.sport.hockey,As the subject suggests the Flames were not im...,18,"[[0.004719207063317299, 0.036944739520549774, ...","[<s>, As, the, subject, suggests, the, Flames,...",0,"[566229, 223, 3, 918, 4650, 3, 20021, 88, 36, ..."
4,19299,rec.sport.hockey,Well I got ta tell ya last night 's Leafs game...,18,"[[-0.0014236271381378174, 0.02442687749862671,...","[<s>, Well, I, got, ta, tell, ya, last, night,...",0,"[566229, 700, 13, 219, 19241, 515, 3619, 190, ..."
...,...,...,...,...,...,...,...,...
10548,20797,alt.atheism,When the object of their belief is said to be ...,13,"[[0.06133417785167694, 0.10471037030220032, 0....","[<s>, When, the, object, of, their, belief, is...",0,"[566229, 257, 3, 2356, 6, 59, 4728, 11, 108, 5..."
10549,20537,alt.atheism,Koff You mean that as long as I put you to sle...,13,"[[-0.04543164372444153, 0.006741240620613098, ...","[<s>, Koff, You, mean, that, as, long, as, I, ...",0,"[566229, 304437, 94, 616, 16, 29, 229, 29, 13,..."
10550,20757,alt.atheism,Mr Connor 's assertion that more complex later...,13,"[[-0.027814187109470367, 0.055848635733127594,...","[<s>, Mr, Connor, 's, assertion, that, more, c...",0,"[566229, 2558, 17914, 21, 18373, 16, 51, 1988,..."
10551,20677,alt.atheism,excess stuff deleted I know of a similar incid...,13,"[[0.01678074523806572, 0.029153533279895782, 0...","[<s>, excess, stuff, deleted, I, know, of, a, ...",0,"[566229, 5814, 809, 6562, 13, 114, 6, 7, 887, ..."


In [64]:
# train_df = train_df.drop(['glove_emb'],axis=1)
# train_df

,id,category,preprocess_text,cat_id,roberta_emb,roberta_tok,glove_int
0,19313,rec.sport.hockey,Kovalev is too talented a player to play for R...,18,"[[-0.1072988361120224, 0.006329931318759918, -...","[<s>, Kovalev, is, too, talented, a, player, t...","[566229, 119013, 11, 188, 5061, 7, 983, 5, 331..."
1,19312,rec.sport.hockey,First of all the Penguins WILL win the cup aga...,18,"[[-0.0944674089550972, 0.004107087850570679, -...","[<s>, First, of, all, the, Penguins, WILL, win...","[566229, 586, 6, 44, 3, 18799, 5777, 814, 3, 2..."
2,19115,rec.sport.hockey,You ca n't But good luck trying,18,"[[-0.05532881245017052, 0.02691790461540222, 0...","[<s>, You, ca, n't, But, good, luck, trying, <...","[566229, 94, 324, 41, 175, 113, 2455, 553, 437..."
3,19224,rec.sport.hockey,As the subject suggests the Flames were not im...,18,"[[0.004719207063317299, 0.036944739520549774, ...","[<s>, As, the, subject, suggests, the, Flames,...","[566229, 223, 3, 918, 4650, 3, 20021, 88, 36, ..."
4,19299,rec.sport.hockey,Well I got ta tell ya last night 's Leafs game...,18,"[[-0.0014236271381378174, 0.02442687749862671,...","[<s>, Well, I, got, ta, tell, ya, last, night,...","[566229, 700, 13, 219, 19241, 515, 3619, 190, ..."
...,...,...,...,...,...,...,...
10548,20797,alt.atheism,When the object of their belief is said to be ...,13,"[[0.06133417785167694, 0.10471037030220032, 0....","[<s>, When, the, object, of, their, belief, is...","[566229, 257, 3, 2356, 6, 59, 4728, 11, 108, 5..."
10549,20537,alt.atheism,Koff You mean that as long as I put you to sle...,13,"[[-0.04543164372444153, 0.006741240620613098, ...","[<s>, Koff, You, mean, that, as, long, as, I, ...","[566229, 304437, 94, 616, 16, 29, 229, 29, 13,..."
10550,20757,alt.atheism,Mr Connor 's assertion that more complex later...,13,"[[-0.027814187109470367, 0.055848635733127594,...","[<s>, Mr, Connor, 's, assertion, that, more, c...","[566229, 2558, 17914, 21, 18373, 16, 51, 1988,..."
10551,20677,alt.atheism,excess stuff deleted I know of a similar incid...,13,"[[0.01678074523806572, 0.029153533279895782, 0...","[<s>, excess, stuff, deleted, I, know, of, a, ...","[566229, 5814, 809, 6562, 13, 114, 6, 7, 887, ..."


In [72]:
train_df.to_csv('./data/20news-bydate-v3/train_df.csv',index=False)
train_df

,id,category,preprocess_text,cat_id,roberta_emb,roberta_tok,glove_int
0,19313,rec.sport.hockey,Kovalev is too talented a player to play for R...,18,"[[-0.1072988361120224, 0.006329931318759918, -...","[<s>, Kovalev, is, too, talented, a, player, t...","[566229, 119013, 11, 188, 5061, 7, 983, 5, 331..."
1,19312,rec.sport.hockey,First of all the Penguins WILL win the cup aga...,18,"[[-0.0944674089550972, 0.004107087850570679, -...","[<s>, First, of, all, the, Penguins, WILL, win...","[566229, 586, 6, 44, 3, 18799, 5777, 814, 3, 2..."
2,19115,rec.sport.hockey,You ca n't But good luck trying,18,"[[-0.05532881245017052, 0.02691790461540222, 0...","[<s>, You, ca, n't, But, good, luck, trying, <...","[566229, 94, 324, 41, 175, 113, 2455, 553, 437..."
3,19224,rec.sport.hockey,As the subject suggests the Flames were not im...,18,"[[0.004719207063317299, 0.036944739520549774, ...","[<s>, As, the, subject, suggests, the, Flames,...","[566229, 223, 3, 918, 4650, 3, 20021, 88, 36, ..."
4,19299,rec.sport.hockey,Well I got ta tell ya last night 's Leafs game...,18,"[[-0.0014236271381378174, 0.02442687749862671,...","[<s>, Well, I, got, ta, tell, ya, last, night,...","[566229, 700, 13, 219, 19241, 515, 3619, 190, ..."
...,...,...,...,...,...,...,...
10548,20797,alt.atheism,When the object of their belief is said to be ...,13,"[[0.06133417785167694, 0.10471037030220032, 0....","[<s>, When, the, object, of, their, belief, is...","[566229, 257, 3, 2356, 6, 59, 4728, 11, 108, 5..."
10549,20537,alt.atheism,Koff You mean that as long as I put you to sle...,13,"[[-0.04543164372444153, 0.006741240620613098, ...","[<s>, Koff, You, mean, that, as, long, as, I, ...","[566229, 304437, 94, 616, 16, 29, 229, 29, 13,..."
10550,20757,alt.atheism,Mr Connor 's assertion that more complex later...,13,"[[-0.027814187109470367, 0.055848635733127594,...","[<s>, Mr, Connor, 's, assertion, that, more, c...","[566229, 2558, 17914, 21, 18373, 16, 51, 1988,..."
10551,20677,alt.atheism,excess stuff deleted I know of a similar incid...,13,"[[0.01678074523806572, 0.029153533279895782, 0...","[<s>, excess, stuff, deleted, I, know, of, a, ...","[566229, 5814, 809, 6562, 13, 114, 6, 7, 887, ..."


### dimension normalize

In [ ]:
train_df.roberta_emb

In [ ]:
kk = train_df.roberta_emb.tolist()
np.array(kk).shape

# Deprecated & Test
* Playground

法2
* deprecated

In [15]:
glove = pd.read_table('./data/preprocess/glove.840B.300d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
# train_df['glove_emb'] = 0
# train_df=train_df.astype(object)

def vec(w):
    return glove.loc[w].values

def glove_convert(tok_li):
    '''
    Input: list of tokens by RoBERTa
    Return: np array of max_length*300dim
    '''
    max_length = 498 #改
    glove_pad = [0]*300
    glove_emb = []
    for token in tok_li:
        glove_emb.append(glove.loc[token].values)
    if len(glove_emb)<max_length:
        for _ in range(max_length-len(glove_emb)):
            glove_emb.append(glove_pad)
    try:
        assert len(glove_emb) == max_length
    except:
        print("maxlength ERR(now,ori):",len(glove_emb) , max_length)
    return np.array(glove_emb)


In [23]:
train_df['glove_emb'] = train_df['roberta_tok'].swifter.apply(glove_convert)
train_df

KeyboardInterrupt: 

In [16]:
# ddata = dd.from_pandas(train_df, npartitions=70)
# res = ddata.map_partitions(lambda df: df['roberta_tok'].apply(glove_convert).compute(get=get))
# res

ValueError: Metadata inference failed in `lambda`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
AttributeError("'Series' object has no attribute 'compute'")

Traceback:
---------
  File "/home/leoqaz12/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py", line 170, in raise_on_meta_error
    exc_type, exc_value, exc_traceback = sys.exc_info()
  File "/home/leoqaz12/anaconda3/lib/python3.7/site-packages/dask/dataframe/core.py", line 4726, in _emulate
    @insert_meta_param_description
  File "<ipython-input-16-9c1047120920>", line 2, in <lambda>
    res = ddata.map_partitions(lambda df: df['roberta_tok'].apply(glove_convert).compute(get=get))
  File "/home/leoqaz12/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py", line 517

In [22]:
# train_df['glove_emb'] = 0
# train_df=train_df.astype(object)
# pandarallel.initialize(shm_size_mb=8192) #shm_size_mb=8192,progress_bar=True
# train_df['glove_emb'] = train_df['roberta_tok'].parallel_apply(glove_convert) #swifter.apply
# train_df

New pandarallel memory created - Size: 8192 MB
Pandarallel will run on 36 workers


ArrowIOError: Connection reset by peer

In [12]:
train_df

,id,category,preprocess_text,cat_id,roberta_emb,roberta_tok,glove_emb
0,19313,rec.sport.hockey,Kovalev is too talented a player to play for R...,18,"[[-0.1072988361120224, 0.006329931318759918, -...","[<s>, Kovalev, is, too, talented, a, player, t...",0
1,19312,rec.sport.hockey,First of all the Penguins WILL win the cup aga...,18,"[[-0.0944674089550972, 0.004107087850570679, -...","[<s>, First, of, all, the, Penguins, WILL, win...",0
2,19115,rec.sport.hockey,You ca n't But good luck trying,18,"[[-0.05532881245017052, 0.02691790461540222, 0...","[<s>, You, ca, n't, But, good, luck, trying, <...",0
3,19224,rec.sport.hockey,As the subject suggests the Flames were not im...,18,"[[0.004719207063317299, 0.036944739520549774, ...","[<s>, As, the, subject, suggests, the, Flames,...",0
4,19299,rec.sport.hockey,Well I got ta tell ya last night 's Leafs game...,18,"[[-0.0014236271381378174, 0.02442687749862671,...","[<s>, Well, I, got, ta, tell, ya, last, night,...",0
...,...,...,...,...,...,...,...
10548,20797,alt.atheism,When the object of their belief is said to be ...,13,"[[0.06133417785167694, 0.10471037030220032, 0....","[<s>, When, the, object, of, their, belief, is...",0
10549,20537,alt.atheism,Koff You mean that as long as I put you to sle...,13,"[[-0.04543164372444153, 0.006741240620613098, ...","[<s>, Koff, You, mean, that, as, long, as, I, ...",0
10550,20757,alt.atheism,Mr Connor 's assertion that more complex later...,13,"[[-0.027814187109470367, 0.055848635733127594,...","[<s>, Mr, Connor, 's, assertion, that, more, c...",0
10551,20677,alt.atheism,excess stuff deleted I know of a similar incid...,13,"[[0.01678074523806572, 0.029153533279895782, 0...","[<s>, excess, stuff, deleted, I, know, of, a, ...",0


Process ForkPoolWorker-71:
Process ForkPoolWorker-72:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/leoqaz12/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/home/leoqaz12/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/home/leoqaz12/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leoqaz12/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leoqaz12/anaconda3/lib/python3.7/site-packages/multiprocess/pool.py", line 110, in worker
    task = get()
  File "/home/leoqaz12/anaconda3/lib/python3.7/site-packages/multiprocess/pool.py", line 110, in worker
    task = get()
  File "/home/leoqaz12/anaconda3/lib/python3.7/site-packages/multiprocess/queues.py"

In [18]:
train_df=train_df.astype(object)
# train_df['roberta_emb'] = train_df.roberta_emb.astype(object)
train_df.loc[i,'roberta_emb'] = np.array(roberta_emb)

In [10]:
train_df.roberta_emb[0].shape

(481, 1024)

In [23]:
type(train_df.id[0])

int

In [ ]:
train_df['roberta_emb'], train_df['roberta_tok'] = zip(*train_df['preprocess_text'].map(tokenize_text))
train_df

In [1]:
train_df

NameError: name 'train_df' is not defined